In [1]:
import pandas as pd
import math

In [2]:
FORWARD_SELECTION, BACKWARD_ELIMINATION = 'forward_selection', 'backward_elimination'
K_FOLD = 10

In [3]:
def read_dataset(dataset_path):

    data = pd.read_csv(dataset_path, delim_whitespace=True, header = None)
    instance_count, column_count = data.shape   
    feature_count = column_count - 1
    instances = data.values.tolist()
    
    return instances, instance_count, feature_count

In [4]:
def find_euclidean_distance(instance, compare_instance, features):
    
    squares = 0
    for feature in features:
        
        diff = instance[feature] - compare_instance[feature]
        squares += diff ** 2
    
    return math.sqrt(squares)

In [5]:
def find_nearest_neighbor(dataset, feature_set, instance_count):
    
    correct_prediction = 0
    for instance_idx in range(instance_count):
        
        instance = dataset[instance_idx]
        target = instance[0]
        features = feature_set 
        
        nearest_neighbor_distance = math.inf
        nearest_neighbor_predict = -1
        
        for compare_idx in range(instance_count):
            
            if compare_idx != instance_idx:
                
                compare_instance = dataset[compare_idx]
                compare_target = compare_instance[0]
                distance = find_euclidean_distance(instance,
                                                  compare_instance,
                                                  features)
                
                if distance < nearest_neighbor_distance:
                        nearest_neighbor_distance = distance
                        nearest_neighbor_predict = compare_target
         
        if nearest_neighbor_predict == target:
            correct_prediction += 1
        
    return correct_prediction / instance_count
            

In [6]:
def leave_one_out_cross_validation(instances, instance_count, feature_set):
    
    
    fold_size = instance_count//K_FOLD
    accuracy_list = []
    for k_fold_itr in range(1, K_FOLD):
        
        ## dataset
        remove_start = (k_fold_itr - 1) * fold_size
        remove_end = remove_start + fold_size
        
        dataset = instances[0:remove_start] + instances[remove_end: instance_count]
        
        accuracy = find_nearest_neighbor(dataset, feature_set, fold_size)
        accuracy_list.append(accuracy)
    
    return sum(accuracy_list)/K_FOLD
        
        

In [7]:
def start_experiment(instances, instance_count, feature_count, search_type = FORWARD_SELECTION):
    
    print(search_type)
    
    feature_set = list(range(1, feature_count + 1))
    accuracy = leave_one_out_cross_validation(\
                    instances, instance_count, feature_set)
    print('Running nearest neighbor with all', feature_count, 'features, using leave-one-out evaluation',
         'with accuracy', '{:0.2f}%'.format(accuracy * 100))
    
    current_feature_set = []
    
    print()
    print('Beginning Search ')
    best_accuracy_feature_set = feature_set
    best_accuracy = accuracy
    for level in range(1, feature_count - 1):
        
        print('On the ', str(level), 'th level of the search tree')
        level_wise_best_accuracy_feature_set = None
        level_wise_best_accuracy_feature = None
        level_wise_best_accuracy = 0
        
        for feature in range(1, feature_count - 1):
            
            if feature not in current_feature_set:
                
                feature_set = current_feature_set + [feature]
                accuracy = leave_one_out_cross_validation(\
                                instances, instance_count, \
                                feature_set)
                
                feature_set_string = ','.join(str(f) for f in feature_set)
                print('\t Using features',  feature_set_string, 'accuracy is', '{:0.2f}%'.format(accuracy * 100) )
                
                if accuracy > level_wise_best_accuracy:
                    level_wise_best_accuracy = accuracy
                    level_wise_best_accuracy_feature = feature
                    level_wise_best_accuracy_feature_set = feature_set
                    level_wise_best_accuracy_feature_set.sort()
                    
        current_feature_set.append(level_wise_best_accuracy_feature)
        level_wise_best_feature_set_string = ','.join(str(f) for f in level_wise_best_accuracy_feature_set)
        print('Feature set ', level_wise_best_feature_set_string, \
              ' was best. accuracy is', '{:0.1f}%'.format(level_wise_best_accuracy*100) )
        print()
        
        if level_wise_best_accuracy > best_accuracy:
            best_accuracy = level_wise_best_accuracy
            best_accuracy_feature_set = level_wise_best_accuracy_feature_set
    
    best_accuracy_feature_set_string = ','.join(str(f) for f in best_accuracy_feature_set)
    print('Finished search !!', 'The best feature subset is', best_accuracy_feature_set_string, \
             'which has an accuracy of', '{:0.1f}%'.format(best_accuracy*100) )

In [8]:
# driver function
if __name__ == "__main__":
    
    small_dataset_path = 'CS205_SP_2022_SMALLtestdata__35.txt'
    large_dataset_path = 'CS205_SP_2022_Largetestdata__62.txt'
    
    while True:

        input_case = input('Press 11 to run Forward Selection with small dataset\n' + \
                          'Press 12 to run Forward Selection with large dataset\n' + \
                          'Press 21 to run Backward Elimination with small dataset\n' + \
                          'Press 22 to run Backward Elimination with large dataset\n' + \
                          'Press any other key to exit\n').strip()
        
        if input_case == '11':
            print("Forward Selection selected with small dataset")
            instances, instance_count, feature_count = read_dataset(small_dataset_path)
            
            start_experiment(instances, instance_count, feature_count, FORWARD_SELECTION)
            
        elif input_case == '12':
            print("Forward Selection selected with large dataset")
            instances, instance_count, feature_count = read_dataset(large_dataset_path)
            
            start_experiment(instances, instance_count, feature_count, FORWARD_SELECTION)
            
        elif input_case == '21':
            print('Backward Elimination selected with small dataset')
            instances, instance_count, feature_count = read_dataset(small_dataset_path)
            
            start_experiment(instances, instance_count, feature_count, BACKWARD_ELIMINATION)
            
        elif input_case == '22':
            print('Backward Elimination selected with large dataset')
            
            instances, instance_count, feature_count = read_dataset(large_dataset_path)
            
            start_experiment(instances, instance_count, feature_count, BACKWARD_ELIMINATION)
        else:
            print('Exit. Thank you')
            break
            

Press 11 to run Forward Selection with small dataset
Press 12 to run Forward Selection with large dataset
Press 21 to run Backward Elimination with small dataset
Press 22 to run Backward Elimination with large dataset
Press any other key to exit
11
Forward Selection selected with small dataset
forward_selection
Running nearest neighbor with all 10 features, using leave-one-out evaluation with accuracy 65.33

On the  1 th level of the search tree
	 Using features 1 accuracy is 71.33
	 Using features 2 accuracy is 70.33
	 Using features 3 accuracy is 71.67
	 Using features 4 accuracy is 68.00
	 Using features 5 accuracy is 65.00
	 Using features 6 accuracy is 65.00
	 Using features 7 accuracy is 65.00
	 Using features 8 accuracy is 75.00
Feature set  8  was best. accuracy is 75.0

On the  2 th level of the search tree
	 Using features 8,1 accuracy is 75.00
	 Using features 8,2 accuracy is 80.67
	 Using features 8,3 accuracy is 78.67
	 Using features 8,4 accuracy is 76.67
	 Using features

KeyboardInterrupt: Interrupted by user